## timm

pa timm module ka da vision dapara dear pretrained models mojoda di. da da rakam di kla nlp dapara shi huggingface di 

aw pa dy notebook ki muzh da maloma wo shi kam model sha di

## The data

pa csv ki da tolo models bench mark mojoda di

In [1]:
! git clone --depth 1 https://github.com/rwightman/pytorch-image-models.git
%cd pytorch-image-models/results

Cloning into 'pytorch-image-models'...
remote: Enumerating objects: 507, done.
remote: Counting objects: 100% (507/507), done.
remote: Compressing objects: 100% (484/484), done.
remote: Total 507 (delta 208), reused 117 (delta 14), pack-reused 0
Receiving objects: 100% (507/507), 1.15 MiB | 725.00 KiB/s, done.
Resolving deltas: 100% (208/208), done.
/home/ikram/Desktop/fastdl/dl1 v5/pytorch-image-models/results


was da pandas pa zarye sra read kawo

In [3]:
import pandas as pd
df_results = pd.read_csv('results-imagenet.csv')

In [4]:
df_results

,model,top1,top1_err,top5,top5_err,param_count,img_size,crop_pct,interpolation
0,beit_large_patch16_512,88.602,11.398,98.656,1.344,305.67,512,1.000,bicubic
1,beit_large_patch16_384,88.406,11.594,98.606,1.394,305.00,384,1.000,bicubic
2,tf_efficientnet_l2_ns,88.350,11.650,98.650,1.350,480.31,800,0.960,bicubic
3,tf_efficientnet_l2_ns_475,88.232,11.768,98.546,1.454,480.31,475,0.936,bicubic
4,deit3_large_patch16_384_in21ft1k,87.716,12.284,98.512,1.488,304.76,384,1.000,bicubic
...,...,...,...,...,...,...,...,...,...
663,dla46_c,64.872,35.128,86.302,13.698,1.30,224,0.875,bilinear
664,lcnet_050,63.094,36.906,84.382,15.618,1.88,224,0.875,bicubic
665,tf_mobilenetv3_small_minimal_100,62.900,37.100,84.234,15.766,2.04,224,0.875,bilinear
666,tinynet_e,59.856,40.144,81.766,18.234,2.04,106,0.875,bicubic


In [5]:
def get_data(part, col):
    df = pd.read_csv(f'benchmark-{part}-amp-nhwc-pt111-cu113-rtx3090.csv').merge(df_results, on='model')
    df['secs'] = 1. / df[col]
    df['family'] = df.model.str.extract('^([a-z]+?(?:v2)?)(?:\d|_|$)')
    df = df[~df.model.str.endswith('gn')]
    df.loc[df.model.str.contains('in22'),'family'] = df.loc[df.model.str.contains('in22'),'family'] + '_in22'
    df.loc[df.model.str.contains('resnet.*d'),'family'] = df.loc[df.model.str.contains('resnet.*d'),'family'] + 'd'
    return df[df.family.str.contains('^re[sg]netd?|beit|convnext|levit|efficient|vit|vgg')]

In [7]:
df = get_data('infer', 'infer_samples_per_sec')
df

,model,infer_samples_per_sec,infer_step_time,infer_batch_size,infer_img_size,param_count_x,top1,top1_err,top5,top5_err,param_count_y,img_size,crop_pct,interpolation,secs,family
12,levit_128s,21485.80,47.648,1024,224,7.78,76.514,23.486,92.870,7.130,7.78,224,0.900,bicubic,0.000047,levit
13,regnetx_002,17821.98,57.446,1024,224,2.68,68.754,31.246,88.556,11.444,2.68,224,0.875,bicubic,0.000056,regnetx
15,regnety_002,16673.08,61.405,1024,224,3.16,70.256,29.744,89.534,10.466,3.16,224,0.875,bicubic,0.000060,regnety
17,levit_128,14657.83,69.849,1024,224,9.21,78.482,21.518,94.012,5.988,9.21,224,0.900,bicubic,0.000068,levit
18,regnetx_004,14440.03,70.903,1024,224,5.16,72.396,27.604,90.838,9.162,5.16,224,0.875,bicubic,0.000069,regnetx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,beit_large_patch16_384,82.25,3112.330,256,384,305.00,88.406,11.594,98.606,1.394,305.00,384,1.000,bicubic,0.012158,beit
567,resnetv2_152x2_bitm,70.34,2729.533,192,448,236.34,84.510,15.490,97.434,2.566,236.34,448,1.000,bilinear,0.014217,resnetv2
568,resnetv2_101x3_bitm,57.82,2213.862,128,448,387.93,84.444,15.556,97.382,2.618,387.93,448,1.000,bilinear,0.017295,resnetv2
573,beit_large_patch16_512,33.13,2897.670,96,512,305.67,88.602,11.398,98.656,1.344,305.67,512,1.000,bicubic,0.030184,beit


## Inference results

Here's the results for inference performance (see the last section for training performance). In this chart:

- the x axis shows how many seconds it takes to process one image (**note**: it's a log scale)
- the y axis is the accuracy on Imagenet
- the size of each bubble is proportional to the size of images used in testing
- the color shows what "family" the architecture is from.

Hover your mouse over a marker to see details about the model. Double-click in the legend to display just one family. Single-click in the legend to show or hide a family.

**Note**: on my screen, Kaggle cuts off the family selector and some plotly functionality -- to see the whole thing, collapse the table of contents on the right by clicking the little arrow to the right of "*Contents*".

In [10]:
import plotly.express as px
w,h = 1000,800

def show_all(df, title, size):
    return px.scatter(df, width=w, height=h, size=df[size]**2, title=title,
        x='secs',  y='top1', log_x=True, color='family', hover_name='model', hover_data=[size])

In [11]:
show_all(df, 'Inference', 'infer_img_size')

That number of families can be a bit overwhelming, so I'll just pick a subset which represents a single key model from each of the families that are looking best in our plot. I've also separated convnext models into those which have been pretrained on the larger 22,000 category imagenet sample (`convnext_in22`) vs those that haven't (`convnext`). (Note that many of the best performing models were trained on the larger sample -- see the papers for details before coming to conclusions about the effectiveness of these architectures more generally.)

In [12]:
subs = 'levit|resnetd?|regnetx|vgg|convnext.*|efficientnetv2|beit'

In this chart, I'll add lines through the points of each family, to help see how they compare -- but note that we can see that a linear fit isn't actually ideal here! It's just there to help visually see the groups.

In [13]:
def show_subs(df, title, size):
    df_subs = df[df.family.str.fullmatch(subs)]
    return px.scatter(df_subs, width=w, height=h, size=df_subs[size]**2, title=title,
        trendline="ols", trendline_options={'log_x':True},
        x='secs',  y='top1', log_x=True, color='family', hover_name='model', hover_data=[size])

In [14]:
show_subs(df, 'Inference', 'infer_img_size')

From this, we can see that the *levit* family models are extremely fast for image recognition, and clearly the most accurate amongst the faster models. That's not surprising, since these models are a hybrid of the best ideas from CNNs and transformers, so get the benefit of each. In fact, we see a similar thing even in the middle category of speeds -- the best is the ConvNeXt, which is a pure CNN, but which takes advantage of ideas from the transformers literature.

For the slowest models, *beit* is the most accurate -- although we need to be a bit careful of interpreting this, since it's trained on a larger dataset (ImageNet-21k, which is also used for *vit* models).

I'll add one other plot here, which is of speed vs parameter count. Often, parameter count is used in papers as a proxy for speed. However, as we see, there is a wide variation in speeds at each level of parameter count, so it's really not a useful proxy.

(Parameter count may be be useful for identifying how much memory a model needs, but even for that it's not always a great proxy.)

In [15]:
px.scatter(df, width=w, height=h,
    x='param_count_x',  y='secs', log_x=True, log_y=True, color='infer_img_size',
    hover_name='model', hover_data=['infer_samples_per_sec', 'family']
)

## Training results

We'll now replicate the above analysis for training performance. First we grab the data:

In [16]:
tdf = get_data('train', 'train_samples_per_sec')

In [17]:
show_all(tdf, 'Training', 'train_img_size')

...and we'll also look at our chosen subset of models:

In [18]:
show_subs(tdf, 'Training', 'train_img_size')

Finally, we should remember that speed depends on hardware. If you're using something other than a modern NVIDIA GPU, your results may be different. In particular, I suspect that transformers-based models might have worse performance in general on CPUs (although I need to study this more to be sure).